## 2.2 A
### Корреляционный анализ (линейная связь)

In [ ]:
corr = train_df.corr(numeric_only=False) 
# corr['oximetry'].sort_values()
corr_sorted = corr[['oximetry']].reindex(
    corr['oximetry'].abs().sort_values(ascending=False).index
)
corr_sorted

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(corr_sorted, annot=True, cmap='coolwarm')
plt.show()

### ANOVA F-test (feature selection test)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

X = train_df.drop(columns=['oximetry'])
y = train_df['oximetry']

selector = SelectKBest(score_func=f_regression, k='all')
X_new = selector.fit_transform(X, y)

idx = selector.get_support(indices=True)
selected_features = X.columns[idx]

anova_results = pd.DataFrame({
    'Feature': selected_features,
    'F_value': selector.scores_[idx],
    'p_value': selector.pvalues_[idx]
}).sort_values(by='F_value', ascending=False)

print("Shape после отбора:", X_new.shape)
print("Выбранные признаки:", list(selected_features))
print(anova_results)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=anova_results, x='F_value', y='Feature')
plt.xscale('log')
plt.title('ANOVA F-test (log scale)')
plt.show()

### Mutual Information (нелинейная зависимость)

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import pandas as pd
import matplotlib.pyplot as plt

selector = SelectKBest(score_func=mutual_info_regression, k=4)
selector.fit(X, y)

scores = pd.Series(abs(selector.scores_), index=X.columns).sort_values()

plt.figure(figsize=(10,5))
scores.plot(kind='barh')
plt.show()

print(scores.sort_values(ascending=False))

In [ ]:
corr_part = corr_sorted['oximetry'].abs().rename('Pearson_|r|')
anova_part = anova_results.set_index('Feature')['F_value'].rename('ANOVA_F')
mi_part = scores.rename('Mutual_Info')

compare_simple = pd.concat([corr_part, anova_part, mi_part], axis=1)
compare_simple = compare_simple.drop(index='oximetry', errors='ignore')
display(compare_simple)

# === 1️⃣ Нормализация в диапазоне [0,1] для каждой метрики ===
norm = compare_simple.apply(lambda s: (s - s.min()) / (s.max() - s.min()), axis=0)

# === 2️⃣ Добавляем среднее значение для финального сравнения ===
norm['Mean_Score'] = norm.mean(axis=1)

# === 3️⃣ Сортируем по информативности ===
final_display = norm.sort_values('Mean_Score', ascending=False)

final_display.style.format('{:.6f}')

## 2.2 B

In [ ]:
ranked_features = final_display.sort_values('Mean_Score', ascending=False)

display(ranked_features.style.format('{:.6f}'))

plt.figure(figsize=(10,6))
sns.barplot(data=ranked_features.head(10).reset_index(), x='Mean_Score', y='index')
plt.title('Top 10 ')
plt.show()

## 2.2 C
### (C-1b) Zdôvodnenie rozhodnutí pri realizácii

Pri realizácii úlohy som sa rozhodol využiť tri komplementárne prístupy na hodnotenie informatívnosti atribútov:
**Pearsonova korelácia**, **ANOVA F-test** a **Mutual Information**.  
Tieto techniky reprezentujú tri rôzne pohľady na vzťah medzi vstupnými premennými a cieľovou premennou *oximetry*:

- **Pearsonova korelácia** umožňuje zachytiť **lineárnu závislosť** medzi atribútom a cieľovou premennou.
- **ANOVA F-test** zisťuje, či existujú **štatisticky významné rozdiely v priemeroch** medzi skupinami hodnôt a hodnotí lineárnu relevantnosť premenných.
- **Mutual Information** je **nelineárna metóda**, ktorá kvantifikuje množstvo informácie, ktorú jeden atribút poskytuje o inom, a teda odhaľuje aj zložitejšie vzťahy.

Výber týchto troch metód umožnil porovnať lineárne aj nelineárne súvislosti a získať
robustnejší prehľad o dôležitosti atribútov.

Všetky výsledky boli následne **normalizované pomocou min–max transformácie** do intervalu ⟨0, 1⟩,  
aby bolo možné spravodlivo porovnávať hodnoty z rôznych metód, ktoré majú odlišné jednotky a rozsahy.
Z týchto normalizovaných hodnôt bol vypočítaný **priemerný index informatívnosti (Mean Score)**,
ktorý vyjadruje celkovú dôležitosť atribútu naprieč metódami.

Týmto postupom bolo zabezpečené:
- porovnateľnosť výsledkov z rôznych štatistických metód,
- odstránenie vplyvu rozdielnych mierok hodnôt (napr. F-hodnoty vs. korelácia),
- transparentný a reprodukovateľný spôsob zoradenia atribútov podľa dôležitosti.

Takto spracované výsledky umožňujú jednoznačne určiť,
ktoré atribúty majú najväčší vplyv na predikciu *oximetry*
a sú vhodné pre ďalšiu tvorbu predikčných modelov.
